## Baseline model : Linear regression
Here I used a very straightforward and simple linear regression model to provide a baseline for our further training. All the advanced models should have a lower RMSE than the baseline.

First as usual, I imported all the packages.

In [18]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['__output__.json', 'finaltrain.csv', '__results___files', 'custom.css', 'finaltest.csv', '__results__.html']


Read the processed train and test data sets.

In [19]:
train=pd.read_csv("../input/finaltrain.csv",index_col=0)
train.head()

/opt/conda/lib/python3.6/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,day,hour,weekday,direct_lat_diff,direct_lon_diff,Distance,fare-bin
0,4.5,2009-06-15 17:26:21,-73.844311,40.721319,-73.841610,40.712278,1,2009,6,15,17,0,0.009041,0.002701,1.030764,"(0, 5]"
1,16.9,2010-01-05 16:52:16,-74.016048,40.711303,-73.979268,40.782004,1,2010,1,5,16,1,0.070701,0.036780,8.450134,"(15, 20]"
2,5.7,2011-08-18 00:35:00,-73.982738,40.761270,-73.991242,40.750562,2,2011,8,18,0,3,0.010708,0.008504,1.389525,"(05, 10]"
3,7.7,2012-04-21 04:30:42,-73.987130,40.733143,-73.991567,40.758092,1,2012,4,21,4,5,0.024949,0.004437,2.799270,"(05, 10]"
4,5.3,2010-03-09 07:51:00,-73.968095,40.768008,-73.956655,40.783762,1,2010,3,9,7,1,0.015754,0.011440,1.999157,"(05, 10]"


In [20]:
test=pd.read_csv("../input/finaltest.csv",index_col=None)
test.head()

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,day,hour,weekday,Distance
0,2015-01-27 13:08:24.0000002,2015-01-27 13:08:24,-73.973320,40.763805,-73.981430,40.743835,1,2015,1,27,13,1,2.323260
1,2015-01-27 13:08:24.0000003,2015-01-27 13:08:24,-73.986862,40.719383,-73.998886,40.739201,1,2015,1,27,13,1,2.425353
2,2011-10-08 11:53:44.0000002,2011-10-08 11:53:44,-73.982524,40.751260,-73.979654,40.746139,1,2011,10,8,11,5,0.618628
3,2012-12-01 21:12:12.0000002,2012-12-01 21:12:12,-73.981160,40.767807,-73.990448,40.751635,1,2012,12,1,21,5,1.961033
4,2012-12-01 21:12:12.0000003,2012-12-01 21:12:12,-73.966046,40.789775,-73.988565,40.744427,1,2012,12,1,21,5,5.387301


Remember in the previous sections, I have already divided the feature 'pickup_datetime' into 5 parts. Thus, we do not need this feature any more. 
Since we already calculated the haversine. The features 'direct_lat_diff' and 'direct_lon_diff' must be highly correlated to the haversine distance. So I also removed these two features to avoid multicolinearity since they provided redundant information. 
'fare_amount' is the target of prediction so I take it aside. Also we need the feature 'key' of the test set as a indentification when we submit the prediction.

In [ ]:
test_key=test['key']
y_train=train['fare_amount']
x_train=train.drop(['fare_amount','direct_lat_diff','direct_lon_diff','fare-bin','pickup_datetime'],axis=1)
x_test=test.drop(['key','pickup_datetime'],axis=1)
print(x_test.head(5))
print(x_train.head(5))

The features 'year','month','day','hour', and 'weekday' are actually categorical features (the number or the absolute value of this number doesn't have any practical meanings). 
So I created one-hot vectors for those features. 
Besides, in order to avoid the influence from different number scales, I normalized all the other numerical features so that I don't need to worry about 
any biased weight on any features.

In [21]:
train_test = pd.concat([x_train,x_test],axis=0)
for col in ['year','month','day','hour','weekday']:
    train_test[col] = train_test[col].astype(str)

print(train_test.dtypes)

pickup_longitude     float64
pickup_latitude      float64
dropoff_longitude    float64
dropoff_latitude     float64
passenger_count        int64
year                  object
month                 object
day                   object
hour                  object
weekday               object
Distance             float64
dtype: object


In [22]:
train_test=pd.get_dummies(train_test,prefix=['year','month','day','hour','weekday'],columns=['year','month','day','hour','weekday'])
train_test.head(5)

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,Distance,year_2009,year_2010,year_2011,year_2012,year_2013,year_2014,year_2015,month_1,month_10,month_11,month_12,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,day_1,day_10,day_11,day_12,day_13,day_14,day_15,day_16,day_17,day_18,day_19,day_2,day_20,day_21,day_22,...,day_3,day_30,day_31,day_4,day_5,day_6,day_7,day_8,day_9,hour_0,hour_1,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_2,hour_20,hour_21,hour_22,hour_23,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6
0,-73.844311,40.721319,-73.841610,40.712278,1,1.030764,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,-74.016048,40.711303,-73.979268,40.782004,1,8.450134,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,-73.982738,40.761270,-73.991242,40.750562,2,1.389525,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,-73.987130,40.733143,-73.991567,40.758092,1,2.799270,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
4,-73.968095,40.768008,-73.956655,40.783762,1,1.999157,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0


In [23]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
train_test[['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','passenger_count','Distance']] = mms.fit_transform(train_test[['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','passenger_count','Distance']])
train_test.head(5)

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,Distance,year_2009,year_2010,year_2011,year_2012,year_2013,year_2014,year_2015,month_1,month_10,month_11,month_12,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,day_1,day_10,day_11,day_12,day_13,day_14,day_15,day_16,day_17,day_18,day_19,day_2,day_20,day_21,day_22,...,day_3,day_30,day_31,day_4,day_5,day_6,day_7,day_8,day_9,hour_0,hour_1,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_2,hour_20,hour_21,hour_22,hour_23,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6
0,0.516775,0.473398,0.518193,0.471981,0.0,0.002268,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,0.495186,0.472117,0.500918,0.480879,0.0,0.018593,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,0.499373,0.478508,0.499415,0.476866,0.2,0.003057,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,0.498821,0.474911,0.499374,0.477827,0.0,0.006159,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
4,0.501214,0.479370,0.503755,0.481103,0.0,0.004399,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0


Finally, I used a very simple linear regression model to fit the training set and do the prediction. 
After training, we can see the R^2 is 0.591, which means a quite poor performance of a simple linear model. 
And when we submit this prediction, the RMSE on the test set is 5.49. Since this task is to predict the fare of a single taxi ride and the baseline price of 
New York City taxi is 2.5. 5.49 is already a huge number in this task. So we need to use advanced model and try to achieve an improvement. 

In [26]:
from sklearn.linear_model import LinearRegression
clf=LinearRegression()
clf.fit(x_train,y_train)
prediction=clf.predict(x_test)

print(prediction)
print(clf.score(x_train,y_train))

[11.0044151  11.33665072  6.82525851 ... 40.28048863 20.03345748
  8.71223068]
0.590952955299167


In [27]:
submission=pd.DataFrame()
submission['key']=test_key
submission['fare_amount']=prediction
submission.to_csv("Baseline.csv",index=False)
submission.head(20)

,key,fare_amount
0,2015-01-27 13:08:24.0000002,11.004415
1,2015-01-27 13:08:24.0000003,11.336651
2,2011-10-08 11:53:44.0000002,6.825259
3,2012-12-01 21:12:12.0000002,9.053080
4,2012-12-01 21:12:12.0000003,14.298385
5,2012-12-01 21:12:12.0000005,11.865692
6,2011-10-06 12:10:20.0000001,7.922546
7,2011-10-06 12:10:20.0000003,45.469316
8,2011-10-06 12:10:20.0000002,11.302556
9,2014-02-18 15:22:20.0000002,8.453405
